<a href="https://colab.research.google.com/github/sachin26240/cats/blob/main/ECDSAv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECDSA

In [ ]:
from sympy import mod_inverse
INF_POINT = None

class EllipticCurve:
	def __init__(self, p, a, b, n):
		self.p = p
		self.a = a
		self.b = b
		self.n = n


	def addition(self, P1, P2):
		if P1 == INF_POINT:
			return P2
		if P2 == INF_POINT:
			return P1

		(x1, y1) = P1
		(x2, y2) = P2

		if self.equal_modp(x1, x2) and self.equal_modp(y1, -y2):
			return INF_POINT

		if self.equal_modp(x1, x2) and self.equal_modp(y1, y2):
			u = self.reduce_modp((3 * x1 * x1 + self.a) * self.inverse_modp(2 * y1))
		else:
			u = self.reduce_modp((y1 - y2) * self.inverse_modp(x1 - x2))

		v = self.reduce_modp(y1 - u * x1)
		x3 = self.reduce_modp(u * u - x1 - x2)
		y3 = self.reduce_modp(-u * x3 - v)
		return (x3, y3)


	def multiple(self, k, P):
		Q = INF_POINT
		while k != 0:
			if k & 1 != 0:
				Q = self.addition(Q, P)
			P = self.addition(P, P)
			k >>= 1
		return Q


	def is_point_on_curve(self, x, y):
		return self.equal_modp(y * y, x * x * x + self.a * x + self.b)


	# helper functions

	def reduce_modp(self, x):
		return x % self.p


	def equal_modp(self, x, y):
		return self.reduce_modp(x - y) == 0


	def inverse_modp(self, x):
		if self.reduce_modp(x) == 0:
			return None
		return pow(x, p - 2, p)
	
	def reduce_modn(self, x):
		return x % self.n


	def equal_modn(self, x, y):
		return self.reduce_modn(x - y) == 0


	def inverse_modn(self, x):
		if self.reduce_modn(x) == 0:
			return None
		return pow(x, n - 2, n)


p = 6277101735386680763835789423207666416083908700390324961279
a = -3
b = int("64210519E59C80E70FA7E9AB72243049FEB8DEECC146B9B1",16)
n = 6277101735386680763835789423176059013767194773182842284081

P224 = EllipticCurve(p, a, b, n)

Gx = int('188DA80EB03090F67CBF20EB43A18800F4FF0AFD82FF1012',16)
Gy = int('07192B95FFC8DA78631011ED6B24CDD573F977A11E794811',16)
G = (Gx, Gy)

# print(P224.is_point_on_curve(Gx, Gy))

In [ ]:
print('Key Generation:')
print('G:',G)
print('Elliptic Curve: y^2=x^3+',a,'x+',b,'(mod',p,')')

Key Generation:
G: (602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641)
Elliptic Curve: y^2=x^3+ -3 x+ 2455155546008943817740293915197451784769108058161191238065 (mod 6277101735386680763835789423207666416083908700390324961279 )


In [ ]:
#private key
d=115792089210356248762697446949407573530086143415290314195533631308865484340366
# d=3
#public key
Q=P224.multiple(d,G)
print('Private Key:')
print(d)
print('Public Key:')
print(Q)

Private Key:
115792089210356248762697446949407573530086143415290314195533631308865484340366
Public Key:
(1894437280655356826717713267896407922731253829512200347153, 137840055381254155571961790247539874133478230753304358678)


In [ ]:
#signature generation
import hashlib
print('Signature Generation:')
k=691073039
print('k =',k)
m='hello'
print('m =',m)
e = int(hashlib.sha1(m.encode()).hexdigest(),16)
print('Hashed Message:',e)

Signature Generation:
k = 691073039
m = hello
Hashed Message: 975987071262755080377722350727279193143145743181


In [ ]:
#calculate x1,r
x=P224.multiple(k,G)
print('k x G:',x)
x1=x[0]
r=x1%n
print('r =',r)

k x G: (5009391581851167523320515380500867018557261816213238223162, 3491543603844729356402651422986775924176839944973180385451)
r = 5009391581851167523320515380500867018557261816213238223162


In [ ]:
#calculate s
import time
start_time = time.time()
# from sympy import mod_inverse
# inverseK=mod_inverse(k, n)
# dr = d*r
# dr = (e + dr)%n
rd=r*d
s = (e+k+rd)%n
print('s =',s)
print("--- %s seconds ---" % (time.time() - start_time))

s = 985411044657475582579558441369017877737191951107675112240
--- 0.0002818107604980469 seconds ---


In [ ]:
#signature verification
if r<n and s<n and r>0 and s>0:
  w=(s-e)%n
  print('w =',w)
  wG = P224.multiple(w,G)
  rQ = P224.multiple(r,Q)
  rQ = (rQ[0],-rQ[1])          #In EC -(x,y) = (x,-y)
  X =P224.addition(wG,rQ)
  print('X =',X)
  v = X[0]%n
  print('v =',v)

w = 985411043681488511316803360991295527009912757964529369059
X = (5009391581851167523320515380500867018557261816213238223162, 3491543603844729356402651422986775924176839944973180385451)
v = 5009391581851167523320515380500867018557261816213238223162


In [ ]:
if r == v:
  print('Signature Verified')
else:
  print('Signature Not Verified')

Signature Verified


In [ ]:
#step3 time taken
start_time = time.time()
w=(s-e)%n
print(w)
print("--- %s seconds ---" % (time.time() - start_time))

985411043681488511316803360991295527009912757964529369059
--- 0.0004646778106689453 seconds ---


Men in Middle Attack

In [ ]:
m="Hello, I am Sender"
m1="Hello, I am Intuder"
